# Statistical & Machine Learning Approaches for Marketing
### Individual Project
### Alejandra Zambrano

In [1]:
# Loading packages
Sys.setenv(LANG = "en")

# Data processing library
library(data.table)       # Data manipulation
library(dplyr)             # Data manipulation
library(stringr)          # String, text processing
library(vita)             # Quickly check variable importance
library(dataPreparation)  # Data preparation library
library(woeBinning)       # Decision tree–based binning for numerical and categorical variables
library(Boruta)           # Variable selection
library(varhandle)        # Dummy variables

# Machine learning library
library(mlr)          # Machine learning framework
library(caret)         # Data processing and machine learning framework
library(MASS)          # LDA
library(randomForest)  # RF
library(gbm)           # Boosting Tree
library(xgboost)       # XGboost

Warning message:
"package 'data.table' was built under R version 3.6.1"Warning message:
"package 'dplyr' was built under R version 3.6.1"
Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'stringr' was built under R version 3.6.1"Warning message:
"package 'vita' was built under R version 3.6.3"Warning message:
"package 'dataPreparation' was built under R version 3.6.3"Loading required package: lubridate
Warning message:
"package 'lubridate' was built under R version 3.6.1"
Attaching package: 'lubridate'

The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following object is masked from 'package:base':

    date

Loading required pack

## 1. Data Summary and Processing

### 1.1. Data Summary

In [2]:
# Reading the data
train=read.csv("bank_mkt_train.csv")
test_holdout=read.csv("bank_mkt_test.csv")

In [3]:
# Print out to check the data type
str(train)

'data.frame':	7000 obs. of  21 variables:
 $ client_id     : int  2 3 4 5 6 7 8 9 14 15 ...
 $ age           : int  29 39 49 32 29 51 34 52 52 29 ...
 $ job           : Factor w/ 12 levels "admin.","blue-collar",..: 4 11 2 7 1 7 2 8 1 1 ...
 $ marital       : Factor w/ 4 levels "divorced","married",..: 3 2 2 3 3 2 2 2 2 3 ...
 $ education     : Factor w/ 8 levels "basic.4y","basic.6y",..: 4 3 2 7 4 7 1 4 7 7 ...
 $ default       : Factor w/ 2 levels "no","unknown": 1 2 2 1 2 2 1 1 1 1 ...
 $ housing       : Factor w/ 3 levels "no","unknown",..: 1 3 1 3 3 3 3 3 3 3 ...
 $ loan          : Factor w/ 3 levels "no","unknown",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ contact       : Factor w/ 2 levels "cellular","telephone": 2 2 1 1 1 2 1 1 1 1 ...
 $ month         : Factor w/ 10 levels "apr","aug","dec",..: 7 5 8 7 4 5 8 8 8 5 ...
 $ day_of_week   : Factor w/ 5 levels "fri","mon","thu",..: 2 1 4 2 1 4 4 4 3 2 ...
 $ campaign      : int  3 6 2 3 2 1 1 1 3 1 ...
 $ pdays         : int  999 999 999 999 9

In [4]:
train$subscribe <-as.factor(train$subscribe)

In [5]:
# Data distribution in the training set
table(train$subscribe)


   0    1 
6178  822 

In [6]:
# Check missing values
apply(is.na(train), 2, sum)

client_id            age            job        marital      education 
             0              0              0              0              0 
       default        housing           loan        contact          month 
             0              0              0              0              0 
   day_of_week       campaign          pdays       previous       poutcome 
             0              0              0              0              0 
  emp.var.rate cons.price.idx  cons.conf.idx      euribor3m    nr.employed 
             0              0              0              0              0 
     subscribe 
             0

#### Splitting train data in train and valid sets

In [7]:
# Split Train set in train_fit and valid_fit
set.seed(123)

train_idx <- caret::createDataPartition(y=train[, 'subscribe'], p=.8, list=F)
train_fit <- train[train_idx, ]  # Train 80%
valid_fit <- train[-train_idx, ]  # Valid 20%

In [8]:
# Target Variable distribution

# By number
table(train_fit$subscribe)
table(valid_fit$subscribe)

# By percentage
table(train_fit$subscribe) / nrow(train_fit)
table(valid_fit$subscribe) / nrow(valid_fit)


   0    1 
4943  658 


   0    1 
1235  164 


       0        1 
0.882521 0.117479 


        0         1 
0.8827734 0.1172266 

In [9]:
# Check Variable Importance

X <- train_fit[, 2:(ncol(train_fit)-1)]
y <- as.factor(train_fit[, 'subscribe'])
rf_model <- randomForest(X, y, mtry=3, ntree=100, importance=TRUE, seed=1)
pimp_varImp <- PIMP(X, y, rf_model, S=10, parallel=F, seed=123)

In [10]:
# Print out top most important variables
pimp_varImp$VarImp[order(pimp_varImp$VarImp[, 1], decreasing=TRUE), ]

month cons.price.idx      euribor3m   emp.var.rate    nr.employed 
  0.0546758154   0.0395329430   0.0355028416   0.0339159346   0.0243178069 
 cons.conf.idx            age            job        contact       poutcome 
  0.0200227034   0.0071033576   0.0065211232   0.0041376890   0.0036769891 
      previous      education    day_of_week          pdays        marital 
  0.0026716002   0.0025445550   0.0022134119   0.0018791391   0.0012390674 
       default        housing       campaign           loan 
  0.0008515970   0.0004255765   0.0003692931   0.0001599518

### 1.2. Preprocessing Data

#### Adding new variables

In [11]:
# Add new variable month_spring
# Train, valid
train_fit[, 'month_spring'] <- as.factor(as.integer((train_fit$month %in% c('mar', 'apr', 'may'))))
valid_fit[, 'month_spring'] <- as.factor(as.integer((valid_fit$month %in% c('mar', 'apr', 'may'))))

# Test (holdout)
test_holdout[, 'month_spring'] <- as.factor(as.integer((test_holdout$month %in% c('mar', 'apr', 'may'))))

In [12]:
# Add new variable month_summer
# Train, valid
train_fit[, 'month_summer'] <- as.factor(as.integer((train_fit$month %in% c('jun', 'jul', 'aug'))))
valid_fit[, 'month_summer'] <- as.factor(as.integer((valid_fit$month %in% c('jun', 'jul', 'aug'))))

# Test (holdout)
test_holdout[, 'month_summer'] <- as.factor(as.integer((test_holdout$month %in% c('jun', 'jul', 'aug'))))

In [13]:
# Add new variable month_autumn
# Train, valid
train_fit[, 'month_autumn'] <- as.factor(as.integer((train_fit$month %in% c('sep', 'oct', 'nov'))))
valid_fit[, 'month_autumn'] <- as.factor(as.integer((valid_fit$month %in% c('sep', 'oct', 'nov'))))

# Test (holdout)
test_holdout[, 'month_autumn'] <- as.factor(as.integer((test_holdout$month %in% c('sep', 'oct', 'nov'))))

In [14]:
# Add new variable month_winter
# Train, valid
train_fit[, 'month_winter'] <- as.factor(as.integer((train_fit$month %in% c('dec', 'jan', 'feb'))))
valid_fit[, 'month_winter'] <- as.factor(as.integer((valid_fit$month %in% c('dec', 'jan', 'feb'))))

# Test (holdout)
test_holdout[, 'month_winter'] <- as.factor(as.integer((test_holdout$month %in% c('dec', 'jan', 'feb'))))

In [15]:
# Add new variable greater than avg_age
# Train, valid
train_fit[, 'age_ge_mean'] <- as.factor(as.integer((train_fit$age > mean(train_fit$age))))
valid_fit[, 'age_ge_mean'] <- as.factor(as.integer((valid_fit$age > mean(valid_fit$age))))

# Test (holdout)
test_holdout[, 'age_ge_mean'] <- as.factor(as.integer((test_holdout$age > mean(test_holdout$age))))

In [16]:
# Add new variable no_contacted before

# pdays == 999 is a special value
# Train, valid
train_fit[, 'no_contacted'] <- as.factor(as.integer((train_fit$pdays == 999)))
valid_fit[, 'no_contacted'] <- as.factor(as.integer((valid_fit$pdays == 999)))

# Test (holdout)
test_holdout[, 'no_contacted'] <- as.factor(as.integer((test_holdout$pdays == 999)))

In [17]:
# Change value 999 in pdays columns

# Train, valid, test
train_fit$pdays<-ifelse(train_fit$pdays==999,0,train_fit$pdays)
valid_fit$pdays<-ifelse(valid_fit$pdays==999,0,valid_fit$pdays)

# Test (holdout)
test_holdout$pdays<-ifelse(test_holdout$pdays==999,0,test_holdout$pdays)

#### Grouping Categorical Variables

In [18]:
# Grouping 12 job categories using WOE
binning_cat <- woe.binning(train_fit, 'subscribe', 'job')
binning_cat

,Group.2,Group.1,woe,iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
7,blue-collar + services,blue-collar,-47.5709908,0.1574216,136,1644,0.2066869,0.3325915,5.989407e-02
8,blue-collar + services,services,-47.5709908,0.1574216,136,1644,0.2066869,0.3325915,5.989407e-02
1,admin. + management + misc. level neg. + technician,admin.,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
2,admin. + management + misc. level neg. + technician,entrepreneur,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
3,admin. + management + misc. level neg. + technician,housemaid,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
4,admin. + management + misc. level neg. + technician,management,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
5,admin. + management + misc. level neg. + technician,technician,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
6,admin. + management + misc. level neg. + technician,unknown,-0.2936055,0.1574216,365,2750,0.5547112,0.5563423,4.788870e-06
9,misc. level pos.,retired,76.4670146,0.1574216,157,549,0.2386018,0.1110662,9.752272e-02
10,misc. level pos.,self-employed,76.4670146,0.1574216,157,549,0.2386018,0.1110662,9.752272e-02


In [19]:
# Apply the job binning to data
train_fit <- woe.binning.deploy(train_fit, binning_cat, add.woe.or.dum.var='woe')
valid_fit<- woe.binning.deploy(valid_fit, binning_cat, add.woe.or.dum.var='woe')
test_holdout <- woe.binning.deploy(test_holdout, binning_cat, add.woe.or.dum.var='woe')

In [20]:
# Grouping 8 education levels using WOE
binning_cat <- woe.binning(train_fit, 'subscribe', 'education')
binning_cat

Group.2,Group.1,woe,iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
basic.9y + misc. level neg.,basic.6y,-33.033099,0.03830303,95,993,0.1443769,0.2008901,0.0186680772
basic.9y + misc. level neg.,basic.9y,-33.033099,0.03830303,95,993,0.1443769,0.2008901,0.0186680772
misc. level pos. + basic.4y + professional.course + high.school,basic.4y,-4.225928,0.03830303,324,2539,0.4924012,0.5136557,0.0008981982
misc. level pos. + basic.4y + professional.course + high.school,high.school,-4.225928,0.03830303,324,2539,0.4924012,0.5136557,0.0008981982
misc. level pos. + basic.4y + professional.course + high.school,illiterate,-4.225928,0.03830303,324,2539,0.4924012,0.5136557,0.0008981982
misc. level pos. + basic.4y + professional.course + high.school,professional.course,-4.225928,0.03830303,324,2539,0.4924012,0.5136557,0.0008981982
misc. level pos. + basic.4y + professional.course + high.school,unknown,-4.225928,0.03830303,324,2539,0.4924012,0.5136557,0.0008981982
university.degree,university.degree,24.093238,0.03830303,239,1411,0.3632219,0.2854542,0.0187367589


In [21]:
# Apply the education binning to data
train_fit <- woe.binning.deploy(train_fit, binning_cat, add.woe.or.dum.var='woe')
valid_fit<- woe.binning.deploy(valid_fit, binning_cat, add.woe.or.dum.var='woe')
test_holdout <- woe.binning.deploy(test_holdout, binning_cat, add.woe.or.dum.var='woe')

In [22]:
# Grouping 10 months using WOE
binning_cat <- woe.binning(train_fit, 'subscribe', 'month')
binning_cat

,Group.2,Group.1,woe,iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
2,aug + nov + jul + jun + may,aug,-32.20431,0.5032246,434,4499,0.6595745,0.91017601,0.08070450
3,aug + nov + jul + jun + may,jul,-32.20431,0.5032246,434,4499,0.6595745,0.91017601,0.08070450
4,aug + nov + jul + jun + may,jun,-32.20431,0.5032246,434,4499,0.6595745,0.91017601,0.08070450
5,aug + nov + jul + jun + may,may,-32.20431,0.5032246,434,4499,0.6595745,0.91017601,0.08070450
6,aug + nov + jul + jun + may,nov,-32.20431,0.5032246,434,4499,0.6595745,0.91017601,0.08070450
1,apr,apr,64.77420,0.5032246,72,283,0.1094225,0.05725268,0.03379258
7,misc. level pos.,dec,195.89989,0.5032246,152,161,0.2310030,0.03257131,0.38872754
8,misc. level pos.,mar,195.89989,0.5032246,152,161,0.2310030,0.03257131,0.38872754
9,misc. level pos.,oct,195.89989,0.5032246,152,161,0.2310030,0.03257131,0.38872754
10,misc. level pos.,sep,195.89989,0.5032246,152,161,0.2310030,0.03257131,0.38872754


In [23]:
# Apply the month binning to data
train_fit <- woe.binning.deploy(train_fit, binning_cat, add.woe.or.dum.var='woe')
valid_fit<- woe.binning.deploy(valid_fit, binning_cat, add.woe.or.dum.var='woe')
test_holdout <- woe.binning.deploy(test_holdout, binning_cat, add.woe.or.dum.var='woe')

In [24]:
# Grouping day of week using WOE
binning_cat <- woe.binning(train_fit, 'subscribe', 'day_of_week')
binning_cat

Group.2,Group.1,woe,iv.total.final,1,0,col.perc.a,col.perc.b,iv.bins
fri + mon,fri,-14.41520,0.01382095,226,1961,0.3434650,0.3967226,0.0076771909
fri + mon,mon,-14.41520,0.01382095,226,1961,0.3434650,0.3967226,0.0076771909
thu + wed,thu,4.78326,0.01382095,279,1998,0.4240122,0.4042080,0.0009472859
thu + wed,wed,4.78326,0.01382095,279,1998,0.4240122,0.4042080,0.0009472859
tue,tue,15.53348,0.01382095,153,984,0.2325228,0.1990694,0.0051964782


In [25]:
# Apply the day_of_week binning to data
train_fit <- woe.binning.deploy(train_fit, binning_cat, add.woe.or.dum.var='woe')
valid_fit<- woe.binning.deploy(valid_fit, binning_cat, add.woe.or.dum.var='woe')
test_holdout <- woe.binning.deploy(test_holdout, binning_cat, add.woe.or.dum.var='woe')

#### Grouping numerical variables

In [26]:
# Build the discretization for age varibale
bins <- build_bins(dataSet=train_fit, cols="age", n_bins=5, type="equal_width", verbose=F)
# Print out to check
bins

$age
[1] 18.0 32.8 47.6 62.4 77.2 92.0

In [27]:
# Apply age discretization to the data

tmp <- fastDiscretization(dataSet=train_fit, bins=bins, verbose=F)
setDF(tmp); setDF(train_fit)  # Convert data.table to data.frame
train_fit[, paste0('age', '_freq_bin')] <- tmp[, 'age']  # Add new variable

tmp <- fastDiscretization(dataSet=valid_fit, bins=bins, verbose=F)
setDF(tmp); setDF(valid_fit)  # Convert data.table to data.frame
valid_fit[, paste0('age', '_freq_bin')] <- tmp[, 'age']  # Add new variable

tmp <- fastDiscretization(dataSet=test_holdout, bins=bins, verbose=F)
setDF(tmp); setDF(test_holdout)  # Convert data.table to data.frame
test_holdout[, paste0('age', '_freq_bin')] <- tmp[, 'age']  # Add new variable

In [28]:
# Build the discretization for pdays varibale
bins <- build_bins(dataSet=train_fit, cols="pdays", n_bins=5, type="equal_width", verbose=F)
# Print out to check
bins

$pdays
[1]  0.0  4.2  8.4 12.6 16.8 21.0

In [29]:
# Apply pdays discretization to the data

tmp <- fastDiscretization(dataSet=train_fit, bins=bins, verbose=F)
setDF(tmp); setDF(train_fit)  # Convert data.table to data.frame
train_fit[, paste0('pdays', '_freq_bin')] <- tmp[, 'pdays']  # Add new variable

tmp <- fastDiscretization(dataSet=valid_fit, bins=bins, verbose=F)
setDF(tmp); setDF(valid_fit)  # Convert data.table to data.frame
valid_fit[, paste0('pdays', '_freq_bin')] <- tmp[, 'pdays']  # Add new variable

tmp <- fastDiscretization(dataSet=test_holdout, bins=bins, verbose=F)
setDF(tmp); setDF(test_holdout)  # Convert data.table to data.frame
test_holdout[, paste0('pdays', '_freq_bin')] <- tmp[, 'pdays']  # Add new variable

#### Creating dummy variables

In [30]:
# Get the IV and DV list name

# Dependent variable (DV)
dv_list <- c('subscribe')
# Independent variable (IV)
iv_list <- setdiff(colnames(train_fit), dv_list)  # Exclude the target variable
iv_list <- setdiff(iv_list, 'client_id')  # Exclude the client_id

In [31]:
# Pick out categorical and numerical variable
iv_cat_list <- c()  # List to store categorical variable
iv_num_list <- c()  # List to store numerical variable
for (v in iv_list) {
    if (class(train_fit[, v]) == 'factor') {  # Factor == categorical variable
        iv_cat_list <- c(iv_cat_list, v)
    } else {  # Non-factor == numerical variable
        iv_num_list <- c(iv_num_list, v)
    }
}

In [32]:
# Join tables to process data in dummy variables
train_fit$data<- 'train'
valid_fit$data <- 'valid'
test_holdout$data <- 'holdout'
test_holdout$subscribe <- NA  # Create a blank response column on test (holdout data)
data <- rbind(train_fit, valid_fit,test_holdout)  # Combine train, valid and test (holdout) data

In [33]:
# Loop through all categorical variable
# Drop the last column of dummy variable
data_dummy_list <- list()
for (v in iv_cat_list) {
    tmp <- to.dummy(data[, v], v)
    data_dummy_list[[length(data_dummy_list) + 1]] <- tmp[, -ncol(tmp), drop=FALSE]
}
data_dummy <- as.data.frame(do.call(cbind, data_dummy_list))

In [34]:
# Drop categorical variables
data_processed <- data[, setdiff(colnames(data), iv_cat_list)]

# Add dummy vairables
data_processed <- cbind(data_processed, data_dummy)

In [35]:
# Split train and test (holdout) after dummitizing
train_fitprocessed <- data_processed[data_processed$data=='train', ]
valid_fitprocessed <- data_processed[data_processed$data=='valid', ]
test_holdoutprocessed <- data_processed[data_processed$data=='holdout',]

In [36]:
# Remove columns 
delete<-c('woe.job.binned','woe.education.binned','woe.month.binned','woe.day_of_week.binned','data','month_winter.0')

train_fitprocessed <- train_fitprocessed[,!(names(train_fitprocessed) %in% delete)]
valid_fitprocessed <- valid_fitprocessed[,!(names(valid_fitprocessed) %in% delete)]
test_holdoutprocessed <- test_holdoutprocessed[,!(names(test_holdoutprocessed) %in% delete)]

#### Standardize Variables

In [37]:
train_fitprocessedstd <- apply(train_fitprocessed[,-c(1,11)],2,scale)
train_fitprocessedstd <- cbind(train_fitprocessed[,c('client_id','subscribe'),drop=FALSE],train_fitprocessedstd)

valid_fitprocessedstd <- apply(valid_fitprocessed[,-c(1,11)],2,scale)
valid_fitprocessedstd <- cbind(valid_fitprocessed[,c('client_id','subscribe'),drop=FALSE],valid_fitprocessedstd)

test_holdoutprocessedstd <- apply(test_holdoutprocessed[,-c(1,11)],2,scale)
test_holdoutprocessedstd <- cbind(test_holdoutprocessed[,c('client_id','subscribe'),drop=FALSE],test_holdoutprocessedstd)


### 1.3. Variable Selection

In [38]:
FisherScore <- function(basetable, depvar, IV_list) {
  "
  This function calculate the Fisher score of a variable.
  
  Ref:
  ---
  Verbeke, W., Dejaeger, K., Martens, D., Hur, J., & Baesens, B. (2012). New insights into churn prediction in the telecommunication sector: A profit driven data mining approach. European Journal of Operational Research, 218(1), 211-229.
  "
  
  # Get the unique values of dependent variable
  DV <- unique(basetable[, depvar])
  
  IV_FisherScore <- c()
  
  for (v in IV_list) {
    fs <- abs((mean(basetable[which(basetable[, depvar]==DV[1]), v]) - mean(basetable[which(basetable[, depvar]==DV[2]), v]))) /
      sqrt((var(basetable[which(basetable[, depvar]==DV[1]), v]) + var(basetable[which(basetable[, depvar]==DV[2]), v])))
    IV_FisherScore <- c(IV_FisherScore, fs)
  }
  
  return(data.frame(IV=IV_list, fisher_score=IV_FisherScore))
}

varSelectionFisher <- function(basetable, depvar, IV_list, num_select) {
  "
  This function will calculate the Fisher score for all IVs and select the best
  top IVs.

  Assumption: all variables of input dataset are converted into numeric type.
  "
  
  fs <- FisherScore(basetable, depvar, IV_list)  # Calculate Fisher Score for all IVs
  num_select <- min(num_select, ncol(basetable))  # Top N IVs to be selected
  return(as.vector(fs[order(fs$fisher_score, decreasing=T), ][1:num_select, 'IV']))
}

In [39]:
# Calculate Fisher Score for all variable
# Get the IV and DV list
dv_list <- c('subscribe')  # DV list
iv_list <- setdiff(names(train_fitprocessedstd), dv_list)  # IV list excluded DV
iv_list <- setdiff(iv_list, 'client_id')  # Excluded the client_id

fs <- FisherScore(train_fitprocessedstd, dv_list, iv_list)
fs <- fs[order(-fs$fisher_score),]
head(fs)

,IV,fisher_score
9,nr.employed,0.7366040
8,euribor3m,0.6934369
5,emp.var.rate,0.6716586
56,no_contacted.0,0.4556426
64,month.binned.aug_+_nov_+_jul_+_jun_+_may,0.4525455
65,month.binned.misc._level_pos.,0.4336046


In [40]:
# Select top 20 variables according to the Fisher Score
best_fs_var <- varSelectionFisher(train_fitprocessedstd, dv_list, iv_list, num_select=20)
best_fs_var

[1] "nr.employed"                             
 [2] "euribor3m"                               
 [3] "emp.var.rate"                            
 [4] "no_contacted.0"                          
 [5] "month.binned.aug_+_nov_+_jul_+_jun_+_may"
 [6] "month.binned.misc._level_pos."           
 [7] "previous"                                
 [8] "poutcome.nonexistent"                    
 [9] "contact.cellular"                        
[10] "pdays"                                   
[11] "pdays_freq_bin.[0,_4.2["                 
[12] "cons.price.idx"                          
[13] "pdays_freq_bin.[4.2,_8.4["               
[14] "month.may"                               
[15] "default.no"                              
[16] "month.oct"                               
[17] "job.binned.misc._level_pos."             
[18] "month.mar"                               
[19] "job.binned.blue-collar_+_services"       
[20] "month_autumn.0"

In [41]:
# Apply variable selection on validation, test and test (holdout)

# train
train_var_select <- names(train_fitprocessedstd)[names(train_fitprocessedstd) %in% best_fs_var]
train_fitprocessed <- train_fitprocessedstd[, c('client_id', train_var_select, 'subscribe')]

# validation
validation_var_select <- names(valid_fitprocessedstd)[names(valid_fitprocessedstd) %in% best_fs_var]
valid_fitprocessed <- valid_fitprocessedstd[, c('client_id', validation_var_select,'subscribe')]

# test holdout
holdout_var_select <- names(test_holdoutprocessedstd)[names(test_holdoutprocessedstd) %in% best_fs_var]
test_holdoutprocessed <- test_holdoutprocessedstd[, c('client_id', holdout_var_select,'subscribe')]


In [42]:
# Rename the data columns
for (v in colnames(train_fitprocessed)) {
    
    # Fix the column name
    fix_name <- str_replace_all(v, "[^[:alnum:] ]", "_")
    fix_name <- gsub(' +', '', fix_name) 
    
    # Train, valid
    colnames(train_fitprocessed)[colnames(train_fitprocessed) == v] <- fix_name
    colnames(valid_fitprocessed)[colnames(valid_fitprocessed) == v] <- fix_name

    # Test (holdout)
    colnames(test_holdoutprocessed)[colnames(test_holdoutprocessed) == v] <- fix_name
}

In [43]:
# Saving the processed datasets
write.csv(train_fitprocessed, 'train_fitprocessed.csv', row.names=FALSE)
write.csv(valid_fitprocessed, 'valid_fitprocessed.csv', row.names=FALSE)
write.csv(test_holdoutprocessed, 'test_holdoutprocessed.csv', row.names=FALSE)

## 2. Methodology

### 2.1. Random Forest

In [45]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.randomForest", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_fitprocessed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
  makeDiscreteParam('ntree', value=c(100, 250, 500, 750, 1000)),
  makeDiscreteParam('mtry', value=round(sqrt((ncol(train_fitprocessed)-1) * c(0.1, 0.25, 0.5, 1, 2, 4)))),
  makeDiscreteParam('nodesize',value=c(10,20,30,40,50))  
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}


[Tune] Started tuning learner classif.randomForest for parameter set:
             Type len Def               Constr Req Tunable Trafo
ntree    discrete   -   - 100,250,500,750,1000   -    TRUE     -
mtry     discrete   -   -          1,2,3,5,6,9   -    TRUE     -
nodesize discrete   -   -       10,20,30,40,50   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: ntree=100; mtry=1; nodesize=10
[Tune-y] 1: auc.test.mean=0.7258187; time: 0.1 min
[Tune-x] 2: ntree=250; mtry=1; nodesize=10
[Tune-y] 2: auc.test.mean=0.7387135; time: 0.1 min
[Tune-x] 3: ntree=500; mtry=1; nodesize=10
[Tune-y] 3: auc.test.mean=0.7426609; time: 0.3 min
[Tune-x] 4: ntree=750; mtry=1; nodesize=10
[Tune-y] 4: auc.test.mean=0.7498014; time: 0.4 min
[Tune-x] 5: ntree=1000; mtry=1; nodesize=10
[Tune-y] 5: auc.test.mean=0.7517889; time: 0.5 min
[Tune-x] 6: ntree=100; mtry=2; nodesize=10
[Tune-y] 6: auc.test.mean=0.7362109; time: 0.1 min
[Tune-x] 7: ntree=250; mtry=2; nodesize=10
[Tune

[Tune-y] 82: auc.test.mean=0.7611927; time: 0.2 min
[Tune-x] 83: ntree=500; mtry=6; nodesize=30
[Tune-y] 83: auc.test.mean=0.7668271; time: 0.5 min
[Tune-x] 84: ntree=750; mtry=6; nodesize=30
[Tune-y] 84: auc.test.mean=0.7653459; time: 0.7 min
[Tune-x] 85: ntree=1000; mtry=6; nodesize=30
[Tune-y] 85: auc.test.mean=0.7668686; time: 1.0 min
[Tune-x] 86: ntree=100; mtry=9; nodesize=30
[Tune-y] 86: auc.test.mean=0.7652155; time: 0.1 min
[Tune-x] 87: ntree=250; mtry=9; nodesize=30
[Tune-y] 87: auc.test.mean=0.7634693; time: 0.3 min
[Tune-x] 88: ntree=500; mtry=9; nodesize=30
[Tune-y] 88: auc.test.mean=0.7679363; time: 0.6 min
[Tune-x] 89: ntree=750; mtry=9; nodesize=30
[Tune-y] 89: auc.test.mean=0.7659061; time: 0.9 min
[Tune-x] 90: ntree=1000; mtry=9; nodesize=30
[Tune-y] 90: auc.test.mean=0.7669046; time: 1.2 min
[Tune-x] 91: ntree=100; mtry=1; nodesize=40
[Tune-y] 91: auc.test.mean=0.7290554; time: 0.1 min
[Tune-x] 92: ntree=250; mtry=1; nodesize=40
[Tune-y] 92: auc.test.mean=0.7418857; 

In [46]:
# Apply optimal parameters to model
best_rf <- setHyperPars(learner = learner,par.vals = res$x)

# Retrain the model with tbe best hyper-parameters
best_rf <- mlr::train(best_rf,train_task)

In [47]:
# Make prediction on valid data
pred <- predict(best_rf, newdata=valid_fitprocessed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7511652

### 2.2. Logistic Regression

In [48]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5, predict="both")

# Define the model
learner <- makeLearner("classif.logreg", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_fitprocessed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc, setAggregation(mlr::auc, train.mean)))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}

Resampling: cross-validation
Measures:             auc.train   auc.test    
[Resample] iter 1:    0.8025760   0.7655947   
[Resample] iter 2:    0.7817572   0.8458334   
[Resample] iter 3:    0.7934952   0.8032416   
[Resample] iter 4:    0.7965415   0.7776343   
[Resample] iter 5:    0.8016874   0.7637756   


Aggregated Result: auc.test.mean=0.7912159,auc.train.mean=0.7952115




In [49]:
# Retrain the model with tbe best hyper-parameters
best_lr <- mlr::train(best_learner, train_task)

In [50]:
# Make prediction on valid data
pred <- predict(best_lr, newdata=valid_fitprocessed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.8055791

### 2.3. Xtreme Gradient Boosting

In [51]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5)

# Define the model
learner <- makeLearner("classif.xgboost", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_fitprocessed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
  makeDiscreteParam('max_depth', value=c(2,3,4,5,6,7)),
  makeDiscreteParam('eta', value=c(0.001,0.01,0.1)),
  makeDiscreteParam('min_child_weight', value=c(2,4,6)),                
  makeDiscreteParam('nrounds', value=c(400,600)),
  makeDiscreteParam('colsample_bytree', value=c(0.5, 0.6,0.7,0.8,0.9)),
  makeDiscreteParam('gamma',value=c(0,0.1,1)),                  
  makeDiscreteParam('subsample',value = c(0.5, 0.6,0.7,0.8,0.9))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}


[Tune] Started tuning learner classif.xgboost for parameter set:
                     Type len Def              Constr Req Tunable Trafo
max_depth        discrete   -   -         2,3,4,5,6,7   -    TRUE     -
eta              discrete   -   -      0.001,0.01,0.1   -    TRUE     -
min_child_weight discrete   -   -               2,4,6   -    TRUE     -
nrounds          discrete   -   -             400,600   -    TRUE     -
colsample_bytree discrete   -   - 0.5,0.6,0.7,0.8,0.9   -    TRUE     -
gamma            discrete   -   -             0,0.1,1   -    TRUE     -
subsample        discrete   -   - 0.5,0.6,0.7,0.8,0.9   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: max_depth=2; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 1: auc.test.mean=0.7870817; time: 0.1 min
[Tune-x] 2: max_depth=3; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 2: auc.test.me

[Tune-y] 46: auc.test.mean=0.7985986; time: 0.1 min
[Tune-x] 47: max_depth=6; eta=0.01; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 47: auc.test.mean=0.7977344; time: 0.1 min
[Tune-x] 48: max_depth=7; eta=0.01; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 48: auc.test.mean=0.7968240; time: 0.2 min
[Tune-x] 49: max_depth=2; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 49: auc.test.mean=0.7930206; time: 0.1 min
[Tune-x] 50: max_depth=3; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 50: auc.test.mean=0.7922069; time: 0.1 min
[Tune-x] 51: max_depth=4; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 51: auc.test.mean=0.7902924; time: 0.1 min
[Tune-x] 52: max_depth=5; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y]

[Tune-y] 96: auc.test.mean=0.7942371; time: 0.2 min
[Tune-x] 97: max_depth=2; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 97: auc.test.mean=0.7925755; time: 0.1 min
[Tune-x] 98: max_depth=3; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 98: auc.test.mean=0.7955512; time: 0.1 min
[Tune-x] 99: max_depth=4; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 99: auc.test.mean=0.7964840; time: 0.2 min
[Tune-x] 100: max_depth=5; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 100: auc.test.mean=0.7969014; time: 0.2 min
[Tune-x] 101: max_depth=6; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5
[Tune-y] 101: auc.test.mean=0.7966916; time: 0.2 min
[Tune-x] 102: max_depth=7; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.5; gamma=0; subsample=0.5

[Tune-y] 145: auc.test.mean=0.7774918; time: 0.1 min
[Tune-x] 146: max_depth=3; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 146: auc.test.mean=0.7901682; time: 0.1 min
[Tune-x] 147: max_depth=4; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 147: auc.test.mean=0.7910044; time: 0.1 min
[Tune-x] 148: max_depth=5; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 148: auc.test.mean=0.7912537; time: 0.1 min
[Tune-x] 149: max_depth=6; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 149: auc.test.mean=0.7928864; time: 0.1 min
[Tune-x] 150: max_depth=7; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 150: auc.test.mean=0.7926348; time: 0.1 min
[Tune-x] 151: max_depth=2; eta=0.01; min_child_weight=6; nrounds=400; colsample_bytree=0.6; gamma=0; s

[Tune-y] 194: auc.test.mean=0.7887838; time: 0.1 min
[Tune-x] 195: max_depth=4; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 195: auc.test.mean=0.7813781; time: 0.2 min
[Tune-x] 196: max_depth=5; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 196: auc.test.mean=0.7776349; time: 0.2 min
[Tune-x] 197: max_depth=6; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 197: auc.test.mean=0.7757591; time: 0.2 min
[Tune-x] 198: max_depth=7; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 198: auc.test.mean=0.7721522; time: 0.2 min
[Tune-x] 199: max_depth=2; eta=0.001; min_child_weight=6; nrounds=600; colsample_bytree=0.6; gamma=0; subsample=0.5
[Tune-y] 199: auc.test.mean=0.7819969; time: 0.1 min
[Tune-x] 200: max_depth=3; eta=0.001; min_child_weight=6; nrounds=600; colsample_bytree=0.6; gamma=0; subsampl

...

[Tune-x] 7885: max_depth=2; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7885: auc.test.mean=0.7725368; time: 0.1 min
[Tune-x] 7886: max_depth=3; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7886: auc.test.mean=0.7914913; time: 0.1 min
[Tune-x] 7887: max_depth=4; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7887: auc.test.mean=0.7935377; time: 0.1 min
[Tune-x] 7888: max_depth=5; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7888: auc.test.mean=0.7957915; time: 0.2 min
[Tune-x] 7889: max_depth=6; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7889: auc.test.mean=0.7955436; time: 0.2 min
[Tune-x] 7890: max_depth=7; eta=0.001; min_child_weight=2; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7890: auc.test.mean

[Tune-y] 7933: auc.test.mean=0.7921933; time: 0.1 min
[Tune-x] 7934: max_depth=3; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7934: auc.test.mean=0.7910822; time: 0.1 min
[Tune-x] 7935: max_depth=4; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7935: auc.test.mean=0.7908417; time: 0.1 min
[Tune-x] 7936: max_depth=5; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7936: auc.test.mean=0.7922929; time: 0.1 min
[Tune-x] 7937: max_depth=6; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7937: auc.test.mean=0.7881865; time: 0.2 min
[Tune-x] 7938: max_depth=7; eta=0.1; min_child_weight=6; nrounds=400; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7938: auc.test.mean=0.7882565; time: 0.2 min
[Tune-x] 7939: max_depth=2; eta=0.001; min_child_weight=2; nrounds=600; colsample_bytree=0.8; gamma=1

[Tune-x] 7982: max_depth=3; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7982: auc.test.mean=0.7971860; time: 0.1 min
[Tune-x] 7983: max_depth=4; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7983: auc.test.mean=0.7974944; time: 0.2 min
[Tune-x] 7984: max_depth=5; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7984: auc.test.mean=0.7986402; time: 0.2 min
[Tune-x] 7985: max_depth=6; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7985: auc.test.mean=0.7975070; time: 0.3 min
[Tune-x] 7986: max_depth=7; eta=0.01; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7986: auc.test.mean=0.7983398; time: 0.3 min
[Tune-x] 7987: max_depth=2; eta=0.1; min_child_weight=6; nrounds=600; colsample_bytree=0.8; gamma=1; subsample=0.9
[Tune-y] 7987: auc.test.mean=0.7906

[Tune-y] 8030: auc.test.mean=0.7903076; time: 0.1 min
[Tune-x] 8031: max_depth=4; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8031: auc.test.mean=0.7940104; time: 0.1 min
[Tune-x] 8032: max_depth=5; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8032: auc.test.mean=0.7942037; time: 0.1 min
[Tune-x] 8033: max_depth=6; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8033: auc.test.mean=0.7949888; time: 0.2 min
[Tune-x] 8034: max_depth=7; eta=0.001; min_child_weight=6; nrounds=400; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8034: auc.test.mean=0.7958956; time: 0.2 min
[Tune-x] 8035: max_depth=2; eta=0.01; min_child_weight=6; nrounds=400; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8035: auc.test.mean=0.7934504; time: 0.1 min
[Tune-x] 8036: max_depth=3; eta=0.01; min_child_weight=6; nrounds=400; colsample_bytree=0.9;

[Tune-x] 8079: max_depth=4; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8079: auc.test.mean=0.7881113; time: 0.2 min
[Tune-x] 8080: max_depth=5; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8080: auc.test.mean=0.7855708; time: 0.2 min
[Tune-x] 8081: max_depth=6; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8081: auc.test.mean=0.7809151; time: 0.2 min
[Tune-x] 8082: max_depth=7; eta=0.1; min_child_weight=4; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8082: auc.test.mean=0.7793711; time: 0.3 min
[Tune-x] 8083: max_depth=2; eta=0.001; min_child_weight=6; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8083: auc.test.mean=0.7725081; time: 0.1 min
[Tune-x] 8084: max_depth=3; eta=0.001; min_child_weight=6; nrounds=600; colsample_bytree=0.9; gamma=1; subsample=0.9
[Tune-y] 8084: auc.test.mean=0.79068

In [52]:
# Apply optimal parameters to model
best_xgb <- setHyperPars(learner = learner,par.vals = res$x)

# Retrain the model with tbe best hyper-parameters
best_xgb <- mlr::train(best_xgb,train_task)

In [53]:
# Make prediction on valid data
pred <- predict(best_xgb, newdata=valid_fitprocessed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7919571

### 2.4. Gradient Boosting

In [62]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=5)

# Define the model
learner <- makeLearner("classif.gbm", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_fitprocessed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
   makeDiscreteParam('n.trees', value=c(300,600,900)),
   makeDiscreteParam('interaction.depth', value=c(3,5,7,9,11)),
   makeDiscreteParam('distribution', value='bernoulli'),
   makeDiscreteParam('shrinkage',value=c(0.1, 0.01, 0.001))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}


[Tune] Started tuning learner classif.gbm for parameter set:
                      Type len Def         Constr Req Tunable Trafo
n.trees           discrete   -   -    300,600,900   -    TRUE     -
interaction.depth discrete   -   -     3,5,7,9,11   -    TRUE     -
distribution      discrete   -   -      bernoulli   -    TRUE     -
shrinkage         discrete   -   - 0.1,0.01,0.001   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: n.trees=300; interaction.depth=3; distribution=bernoulli; shrinkage=0.1
[Tune-y] 1: auc.test.mean=0.7851264; time: 0.1 min
[Tune-x] 2: n.trees=600; interaction.depth=3; distribution=bernoulli; shrinkage=0.1
[Tune-y] 2: auc.test.mean=0.7827584; time: 0.2 min
[Tune-x] 3: n.trees=900; interaction.depth=3; distribution=bernoulli; shrinkage=0.1
[Tune-y] 3: auc.test.mean=0.7769550; time: 0.3 min
[Tune-x] 4: n.trees=300; interaction.depth=5; distribution=bernoulli; shrinkage=0.1
[Tune-y] 4: auc.test.mean=0.7837652; time: 0.2 min
[T

In [63]:
# Apply optimal parameters to model
best_gb <- setHyperPars(learner = learner,par.vals = res$x)

# Retrain the model with tbe best hyper-parameters
best_gb <- mlr::train(best_gb,train_task)

In [64]:
# Make prediction on valid data
pred <- predict(best_gb, newdata=valid_fitprocessed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7995902

### 2.5. k-Nearest Neighbor

In [47]:
# Set up cross-validation
rdesc = makeResampleDesc("CV", iters=10)

# Define the model
learner <- makeLearner("classif.kknn", predict.type="prob", fix.factors.prediction=T)

# Define the task
train_task <- makeClassifTask(id="bank_train", data=train_fitprocessed[, -1], target="subscribe")

# Set hyper parameter tuning
tune_params <- makeParamSet(
  makeDiscreteParam('k', value=c(1:30))
)
ctrl = makeTuneControlGrid()

# Run the hyper parameter tuning with k-fold CV
if (length(tune_params$pars) > 0) {
    # Run parameter tuning
    res <- tuneParams(learner, task=train_task, resampling=rdesc,
      par.set=tune_params, control=ctrl, measures=list(mlr::auc))
    
    # Extract best model
    best_learner <- res$learner
    
} else {
    # Simple cross-validation
    res <- resample(learner, train_task, rdesc, measures=list(mlr::auc))
    
    # No parameter for tuning, only 1 best learner
    best_learner <- learner
}


Loading required package: kknn
Warning message:
"package 'kknn' was built under R version 3.6.2"
Attaching package: 'kknn'

The following object is masked from 'package:caret':

    contr.dummy

[Tune] Started tuning learner classif.kknn for parameter set:
      Type len Def                                   Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,1...   -    TRUE     -
With control class: TuneControlGrid
Imputation value: -0
[Tune-x] 1: k=1
[Tune-y] 1: auc.test.mean=0.6251465; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: auc.test.mean=0.6741682; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: auc.test.mean=0.7069733; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: auc.test.mean=0.7218334; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: auc.test.mean=0.7269256; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: auc.test.mean=0.7315156; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: auc.test.mean=0.7358139; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: auc.test.mean=0.7421720; time: 0.

In [48]:
# Apply optimal parameters to model
best_kn <- setHyperPars(learner = learner,par.vals = res$x)

# Retrain the model with tbe best hyper-parameters
best_kn <- mlr::train(best_kn,train_task)

In [49]:
# Make prediction on valid data
pred <- predict(best_kn, newdata=valid_fitprocessed[, -1])
performance(pred, measures=mlr::auc)

auc 
0.7708996